# README
This cookbook was created by Joe Constantino on June 25, 2024 to test an NLQ-to-VDS query pipeline for answering ad-hoc questions over Tableau Published Data Sources

This cookbook can be configured to query a Tableau Published data source in natural language using the Headless BI REST endpoint. It uses langchain libraries and the langchain development framework to generate a headless BI request payload from a natural language input query. It then executes a chain to generate an answer to the user's input query.

To get started with this notebook, make sure you've added your environment variables to your .env file. The environment variables are set up to consume a PAT token associated to a specific Tableau site. More info on authenticating to VizQL Data Service can be found here: https://help.tableau.com/v0.0/api/vizql-data-service/en-us/docs/vds_setup.html

We recommend using gpt-4o or gpt-4o-mini for the VDS agent chain.

In [1]:
#import all required libraries
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import pandas as pd
import os
import json
import requests
import re
from dotenv import load_dotenv


from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

# loading environment variables
env_vars = os.environ

In [2]:
#try read in the datasource metadata of the datasource that is defined in your environment variables using the os.environ command.
#If this cell fails, proceed to next cell
def read():
    url = env_vars['READ_METADATA']
    payload = json.dumps({
        "connection": {
            "tableauServerName": env_vars['TABLEAU_DOMAIN'],
            "siteId": env_vars['SITE_NAME'],
            "datasource": env_vars['DATA_SOURCE']
        },
    })

    headers = {
    'Credential-Key': env_vars['PAT_NAME'],
    'Credential-value': env_vars['PAT_SECRET'],
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        #df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return data
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)
datasource_metadata = read()


KeyError: 'READ_METADATA'

In [3]:
#read in the datasource metadata of the datasource that is defined in your environment variables using load_dotenv().
# If the above cell succeeded, skip this cell
load_dotenv()
def read():
    url = os.getenv('READ_METADATA')
    payload = json.dumps({
        "connection": {
            "tableauServerName": os.getenv('TABLEAU_DOMAIN'),
            "siteId": os.getenv('SITE_NAME'),
            "datasource": os.getenv('DATA_SOURCE')
        },
    })

    headers = {
    'Credential-Key': os.getenv('PAT_NAME'),
    'Credential-value': os.getenv('PAT_SECRET'),
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        # Create a pandas DataFrame from the JSON data
        #df = pd.DataFrame(data)

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return data
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)
datasource_metadata = read()

In [10]:
#define the headless BI query template
def send(query):
    url = env_vars['VDS_URL']
    payload = json.dumps({
        "connection": {
            "tableauServerName": env_vars['TABLEAU_DOMAIN'],
            "siteId": env_vars['SITE_NAME'],
            "datasource": env_vars['DATA_SOURCE']
        },
        "query": query
    })

    headers = {
    'Credential-Key': env_vars['PAT_NAME'],
    'Credential-value': env_vars['PAT_SECRET'],
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the response data
        #print("Response Data:")
        #print(response.json())
        data = response.json()['data']
        print(data)
        # Create a pandas DataFrame from the JSON data
        df = pd.DataFrame(data)
        df

        # Display the first few rows of the DataFrame
        #print("Table view of data from the public REST API:")
        #print(df.head())
        #print(df.all())
        return df
        #display(df.head())
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        print(response.text)

In [5]:
# Specify the path to the system prompt
path = './prompts/nlq_to_vds.py'

# read in system prompt
with open(path, 'r') as json_file:
    headless_bi_prompt_string = json_file.read()

#remove "prompt = " prefix from the string
stripped_headless_bi_prompt_string = headless_bi_prompt_string.replace('prompt = ', '', 1).strip()

#convert headless bi prompt string to a json object
headless_bi_prompt = json.loads(stripped_headless_bi_prompt_string)

# add the datasource metadata of the connected datasource to the system prompt
headless_bi_prompt['data_model'] = datasource_metadata

# #convert prompt back to a string to pass in LLM request
# ready_prompt = json.dumps(headless_bi_prompt)
# print(ready_prompt)

In [9]:
#clean up and augment data model
for item in headless_bi_prompt['data_model']:
    del item['objectGraphId']

In [10]:
# Print the modified list
headless_bi_prompt['data_model']

[{'columnName': 'AclId (Events)',
  'caption': 'AclId (Events)',
  'dataType': 'STRING',
  'columnContainsNulls': True},
 {'columnName': 'AclId',
  'caption': 'Acl Id',
  'dataType': 'STRING',
  'columnContainsNulls': True},
 {'columnName': 'AnchorDate (Events)',
  'caption': 'AnchorDate (Events)',
  'dataType': 'DATE',
  'columnContainsNulls': True},
 {'columnName': 'AnchorDate',
  'caption': 'Anchor Date',
  'dataType': 'DATE',
  'columnContainsNulls': True},
 {'columnName': 'Calculation_1532349828299501568',
  'caption': 'Start-Form to Event-Start',
  'dataType': 'INTEGER',
  'columnContainsNulls': True},
 {'columnName': 'Calculation_2057863609944670208',
  'caption': 'OnTherapyIndicator',
  'dataType': 'INTEGER',
  'columnContainsNulls': True},
 {'columnName': 'Calculation_2057863609945575425',
  'caption': 'Patient Treated?',
  'dataType': 'INTEGER',
  'columnContainsNulls': True},
 {'columnName': 'Calculation_2057863609986830338',
  'caption': 'Fail to Reach Therapy',
  'dataType

In [11]:
#function to pull in sample values for give column
def augment(column_name):
    if column_name == 'PatientStatus1':
        return ""
    column_values_json = {'columns': [{'columnName': column_name}]}
    output = send(column_values_json)
    if output is None:
        return None
    sample_values = output.values.flatten()[:4]
    return sample_values.tolist()

In [12]:
for field in headless_bi_prompt['data_model']:
    if field['dataType'] == 'STRING':
        string_values = augment(field['columnName'])
        field['sampleValues'] = string_values

In [ ]:
print(headless_bi_prompt['data_model'])

In [20]:
#extract caption and column names to add to the available_fields field in the prompt
avail_fields = []
for item in headless_bi_prompt['data_model']:
    column_name = 'columnName: ' + item['columnName']
    caption = 'caption: ' + item['caption']
    avail_fields.append((column_name, caption))

headless_bi_prompt['available_fields'] = avail_fields

In [21]:
#convert prompt back to a string to pass in LLM request
ready_prompt = json.dumps(headless_bi_prompt)
#print(ready_prompt)

In [6]:
#define function for getting user queries
def get_utterance():
  query = input('what would you like to know about your data? reply stop if you are done.')
  return query


In [7]:
active_utterance = get_utterance()

while (active_utterance != 'stop'):
  llm = ChatOpenAI(model=env_vars['VDS_AGENT_MODEL'], temperature=0)
  active_prompt_template = ChatPromptTemplate.from_messages([
      SystemMessage(content=headless_bi_prompt_string),
      ("user", "{utterance}")])
  prompt_value = active_prompt_template.invoke(
      {
          "utterance": active_utterance
      }
  )
  output_parser = StrOutputParser()
  chain = active_prompt_template | llm | output_parser
  output = chain.invoke(active_utterance)

  #print reasoning
  print(output.split('JSON_payload')[0])

  #parse LLM output and query headless BI
  parsed_output = output.split('JSON_payload')[1]
  match = re.search(r'{.*}', parsed_output, re.DOTALL)
  if match:
    json_string = match.group(0)
    payload = json.loads(json_string)
  print(payload)
  df = send(payload)
  agent = create_pandas_dataframe_agent(
      ChatOpenAI(temperature=0, model=env_vars['PANDAS_AGENT_MODEL']),
      df,
      verbose=True,
      agent_type=AgentType.OPENAI_FUNCTIONS,
      allow_dangerous_code=True
  )
  instruct_header = "answer the following query directly by executing the necessary python code. Give a succinct explanation of the steps you took and how you know the answer is correct: "
  agent.invoke(instruct_header + active_utterance)
  active_utterance = get_utterance()

what would you like to know about your data? reply stop if you are done. test


Reasoning: The user input "test" does not provide enough context to generate a specific VDS query. However, I will create a basic template for a VDS query that can be used as a starting point.


{'columns': [{'columnName': 'SampleColumn'}]}
Failed to fetch data from the API. Status code: 400
{"errorCode":"400002","message":"Column 'SampleColumn' does not exist","datetime":"2024-09-13T23:31:33.900874Z","details":{"errorCode":"500104","message":"Base column [DSInternalName].[SampleColumn] does not exist.","datetime":"2024-09-13T23:31:33.814157Z","details":null}}


ValueError: Expected pandas DataFrame, got <class 'NoneType'>

In [12]:
#test nlq_to_vds chain
test_utterance = "which products tend to have the largest discounts?"
llm = ChatOpenAI(model=env_vars['VDS_AGENT_MODEL'])
active_prompt_template = ChatPromptTemplate.from_messages([
  SystemMessage(content=headless_bi_prompt_string),
  ("user", "{utterance}")])
prompt_value = active_prompt_template.invoke(
  {
      "utterance": test_utterance
  }
)
output_parser = StrOutputParser()
chain = active_prompt_template | llm | output_parser
output = chain.invoke(test_utterance)

#print reasoning
print(output.split('JSON_payload')[0])

#parse LLM output and print generated json
parsed_output = output.split('JSON_payload')[1]
match = re.search(r'{.*}', parsed_output, re.DOTALL)
if match:
    json_string = match.group(0)
json_payload = json.loads(json_string)
print(json_payload)

Reasoning: To determine which products tend to have the largest discounts, we need to retrieve the product information along with their discount values. We can then analyze the discount values to identify the products with the largest discounts. 


{'columns': [{'columnName': 'Product Name'}, {'columnName': 'Discount', 'function': 'AVG', 'sortPriority': 1, 'sortDirection': 'DESC'}]}


In [13]:
#test json
json_to_test = {'columns': [{'columnName': 'Product Name'}, {'columnName': 'Discount', 'function': 'AVG', 'sortPriority': 1, 'sortDirection': 'DESC'}]}
output = send(json_to_test)


[{'Product Name': 'Eureka Disposable Bags for Sanitaire Vibra Groomer I Upright Vac', 'AVG(Discount)': 0.8}, {'Product Name': 'GBC Plasticlear Binding Covers', 'AVG(Discount)': 0.7285714285714285}, {'Product Name': 'GBC VeloBinder Electric Binding Machine', 'AVG(Discount)': 0.7249999999999999}, {'Product Name': 'Zebra GK420t Direct Thermal/Thermal Transfer Printer', 'AVG(Discount)': 0.7}, {'Product Name': 'Cisco 8961 IP Phone Charcoal', 'AVG(Discount)': 0.7}, {'Product Name': 'Epson Perfection V600 Photo Scanner', 'AVG(Discount)': 0.7}, {'Product Name': 'Okidata B401 Printer', 'AVG(Discount)': 0.7}, {'Product Name': 'Hewlett-Packard Deskjet F4180 All-in-One Color Ink-jet - Printer / copier / scanner', 'AVG(Discount)': 0.7}, {'Product Name': 'Brother MFC-9340CDW LED All-In-One Printer, Copier Scanner', 'AVG(Discount)': 0.7}, {'Product Name': 'Lexmark MarkNet N8150 Wireless Print Server', 'AVG(Discount)': 0.7}, {'Product Name': 'Bush Westfield Collection Bookcases, Dark Cherry Finish, Fu

In [37]:
sample_values = output.values.flatten()[:4]
sample_values.tolist()

['B2D7DB439B69BA01FBF3CEC04682CF4BA6CEB02AF0C8ADBF6EE56CD5857F5ADA',
 'D67BBADED27258650B037F097422C823C1F1F8E9A33E03AB235564007785D67E',
 '01DC423A0092DC4E33A1BD15BEF77EFF9771A2BC48306B2F49811D53A4B31060',
 '6C0643B03913C586636C76A29D7C7B9CF978677B4456C59DF29B7E34D3AA1A75']